In [1]:
import dlib

def get_landmark_model():
    # Download the dlib pre-trained facial landmark detector
    # This will download a file called 'shape_predictor_68_face_landmarks.dat'
    # into the current working directory
    dlib_model_url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
    dlib_model_file = 'shape_predictor_68_face_landmarks.dat'
    dlib_model_path = os.path.join("E:\\shape_predictor_68_face_landmarks.dat")
    if not os.path.exists(dlib_model_path):
        print('Downloading dlib facial landmark detection model...')
        urllib.request.urlretrieve(dlib_model_url, dlib_model_path)

    # Load the dlib facial landmark detector
    landmark_model = dlib.shape_predictor(dlib_model_path)

    return landmark_model

In [2]:
import dlib
import cv2
import urllib.request
import os

# Load the landmark model
landmark_model = get_landmark_model()

# Load the image
image = cv2.imread("E:\\Rakshith\\Photos.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect the faces in the image
detector = dlib.get_frontal_face_detector()
faces = detector(gray, 1)

# Loop over the faces and detect the facial landmarks
for face in faces:
    landmarks = landmark_model(gray, face)

In [3]:
import dlib

def get_face_detector():
    # Download the dlib pre-trained face detector
    # This will download a file called 'mmod_human_face_detector.dat'
    # into the current working directory
    dlib_model_url = 'http://dlib.net/files/mmod_human_face_detector.dat.bz2'
    dlib_model_file = 'mmod_human_face_detector.dat'
    dlib_model_path = os.path.join("E:\\mmod_human_face_detector.dat")
    if not os.path.exists(dlib_model_path):
        print('Downloading dlib face detection model...')
        urllib.request.urlretrieve(dlib_model_url, dlib_model_path)

    # Load the dlib face detector
    face_detector = dlib.cnn_face_detection_model_v1(dlib_model_path)

    return face_detector

In [4]:
import dlib
import cv2
import os
import urllib.request

# Load the face detector model
face_detector = get_face_detector()

# Load the image
image = cv2.imread("E:\\Rakshith\\Photos.jpg")

# Detect the faces in the image
detections = face_detector(image, 1)

# Loop over the detections and do something with them
for detection in detections:
    left = detection.rect.left()
    top = detection.rect.top()
    right = detection.rect.right()
    bottom = detection.rect.bottom()

   

In [5]:
import cv2

In [6]:
import dlib

def find_faces(image, face_detector):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale image
    faces = face_detector(gray, 1)
    
    # Return the list of faces as rectangles
    return [(face.left(), face.top(), face.right(), face.bottom()) for face in faces]

    # Return the list of faces
    return faces


In [7]:
import dlib

def find_faces(image, detector):
    face_detector = detector
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = face_detector(gray, 0)
    return rects


In [8]:
def detect_marks(img, model, face):
    offset_y = int(abs((face.rect.bottom() - face.rect.top()) * 0.1))
    box_moved = move_box([face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom()], [0, offset_y])
    facebox = get_square_box(box_moved)

    face_img = img[facebox[1]: facebox[3],
               facebox[0]: facebox[2]]
    try:
        # Converting the face image to grayscale
        face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)

        # Applying Gaussian blur
        face_img = cv2.GaussianBlur(face_img, (5, 5), 0)

        # Resizing the face image to 96x96 for landmark detection
        face_img = cv2.resize(face_img, (96, 96))

        # Normalizing the pixel values
        face_img = face_img / 255.0

        # Adding a batch dimension to the face image
        face_img = np.expand_dims(face_img, axis=0)

        # Detecting landmarks in the face image
        marks = model.predict(face_img)[0]
        
        # Converting the landmarks to (x, y) coordinates
        marks = np.array([(int(x * facebox[2] + facebox[0]), int(y * facebox[3] + facebox[1])) for (x, y) in marks])

        return marks
    except:
        pass


In [9]:
def detect_marks(img, model, face):
    # Check if face is valid
    if face is None:
        return None
    
    # Calculate offset for mouth
    offset_y = int(abs((face.rect.bottom() - face.rect.top()) * 0.1))
    
    # Move the face box down by the offset
    face_box = [face.rect.left(), face.rect.top() + offset_y, face.rect.right(), face.rect.bottom() + offset_y]
    
    # Get square box around face
    face_box = get_square_box(face_box)
    
    # Extract face image
    face_img = img[face_box[1]:face_box[3], face_box[0]:face_box[2]]
    
    # Detect facial landmarks
    marks = []
    try:
        marks = detect_landmarks(face_img, model)
    except:
        print("Failed to detect landmarks")
        return None
    
    # Convert coordinates to global coordinates
    marks = [(mark[0] + face_box[0], mark[1] + face_box[1]) for mark in marks]
    
    return marks


In [10]:
def move_box(box, offset):
    x1, y1, x2, y2 = box
    x1 += offset[0]
    y1 += offset[1]
    x2 += offset[0]
    y2 += offset[1]
    return [x1, y1, x2, y2]


In [11]:
 def get_square_box(box):
        """Get a square box out of the given box, by expanding it."""
        left_x = box[0]
        top_y = box[1]
        right_x = box[2]
        bottom_y = box[3]

        box_width = right_x - left_x
        box_height = bottom_y - top_y

        # Check if box is already a square. If not, make it a square.
        diff = box_height - box_width
        delta = int(abs(diff) / 2)

        if diff == 0:  # Already a square.
            return box
        elif diff > 0:  # Height > width, a slim box.
            left_x -= delta
            right_x += delta
            if diff % 2 == 1:
                right_x += 1
        else:  # Width > height, a short box.
            top_y -= delta
            bottom_y += delta
            if diff % 2 == 1:
                bottom_y += 1

        # Make sure box is always square.
        assert ((right_x - left_x) == (bottom_y - top_y)), 'Box is not square.'

        return [left_x, top_y, right_x, bottom_y]


In [12]:
def draw_marks(image, marks, color=(0, 255, 0)):
    """
    Draws facial landmark points on the image.

    Args:
    image: A numpy array representing an image.
    marks: A numpy array of shape (68, 2) containing the x, y coordinates of facial landmark points.
    color: A tuple representing the color of the points.

    Returns:
    None
    """
    if marks is not None:
        for mark in marks:
            cv2.circle(image, (mark[0], mark[1]), 2, color, -1, cv2.LINE_AA)


In [ ]:
face_model = get_face_detector()
landmark_model = get_landmark_model()
outer_points = [[49, 59], [50, 58], [51, 57], [52, 56], [53, 55]]
d_outer = [0]*5
inner_points = [[61, 67], [62, 66], [63, 65]]
d_inner = [0]*3
font = cv2.FONT_HERSHEY_SIMPLEX 
cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()
    rects = find_faces(img, face_model)
    for rect in rects:
        shape = detect_marks(img, landmark_model, rect)
        draw_marks(img, shape)
        cv2.putText(img, 'Press r to record Mouth distances', (30, 30), font,
                    1, (0, 255, 255), 2)
        cv2.imshow("Output", img)
    if cv2.waitKey(1) & 0xFF == ord('r'):
        for i in range(100):
            for i, (p1, p2) in enumerate(outer_points):
                d_outer[i] += shape[p2][1] - shape[p1][1]
            for i, (p1, p2) in enumerate(inner_points):
                d_inner[i] += shape[p2][1] - shape[p1][1]
        break
cv2.destroyAllWindows()
d_outer[:] = [x / 100 for x in d_outer]
d_inner[:] = [x / 100 for x in d_inner]

while(True):
    ret, img = cap.read()
    rects = find_faces(img, face_model)
    for rect in rects:
        shape = detect_marks(img, landmark_model, rect)
        cnt_outer = 0
        cnt_inner = 0
        draw_marks(img, shape[48:])
        for i, (p1, p2) in enumerate(outer_points):
            if d_outer[i] + 3 < shape[p2][1] - shape[p1][1]:
                cnt_outer += 1 
        for i, (p1, p2) in enumerate(inner_points):
            if d_inner[i] + 2 <  shape[p2][1] - shape[p1][1]:
                cnt_inner += 1
        if cnt_outer > 3 and cnt_inner > 2:
            print('Mouth open')
            cv2.putText(img, 'Mouth open', (30, 30), font,
                    1, (0, 255, 255), 2)
        # show the output image with the face detections + facial landmarks
    cv2.imshow("Output", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

Failed to detect landmarks
Failed to detect landmarks
Failed to detect landmarks
Failed to detect landmarks
Failed to detect landmarks
Failed to detect landmarks
Failed to detect landmarks
Failed to detect landmarks
Failed to detect landmarks
Failed to detect landmarks
Failed to detect landmarks
